In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tc.dataset
import numpy as np
from ase.build import bulk
from mace.calculators import mace_mp
from monty.serialization import dumpfn

Mg_conv_cell = bulk("MgO", crystalstructure="rocksalt", a=4.2, cubic=True)
rng = np.random.default_rng(42)
calc = mace_mp(model="large", device="cuda", default_dtype="float64")
ensemble = tc.dataset.make_ce_ensemble_from_mace(
    conv_cell=Mg_conv_cell,
    rng=rng,
    calc=calc,
    replace_element="Mg",
    new_elements=("Li", "Mn"),
)

dumpfn(ensemble, "LiMnO_ensemble.json.gz", indent=2)

/home/colin/miniforge/envs/tc312/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


Using Materials Project MACE for MACECalculator with /home/colin/.cache/mace/MACE_MPtrj_20229model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/home/colin/miniforge/envs/tc312/lib/python3.12/site-packages/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Total snapshots: 60


MACE energies:   0%|          | 0/60 [00:00<?, ?it/s]

Adding:   0%|          | 0/60 [00:00<?, ?it/s]

Matched structures: 60/60
RMSE     5.01 meV   MAX    10.98 meV
5-fold CV RMSE: 8.48083683738291 meV


In [1]:
from monty.serialization import dumpfn
dumpfn(ensemble, "LiMnO_ensemble.json.gz", indent=2)

NameError: name 'ensemble' is not defined

In [ ]:
import numpy as np
import tc.testing
from monty.serialization import loadfn
from mace.calculators import mace_mp
from ase.build import bulk

ensemble_reloaded = loadfn("LiMnO_ensemble.json.gz")


Mg_conv_cell = bulk("MgO", crystalstructure="rocksalt", a=4.2, cubic=True)
rng = np.random.default_rng(42)
calc = mace_mp(model="large", device="cuda", default_dtype="float64")
stats = tc.testing.evaluate_ensemble_vs_mace(
    ensemble        = ensemble_reloaded,
    calc            = calc,
    conv_cell       = Mg_conv_cell,
    rng             = rng,
    replace_element = "Mg",
    new_elements    = ("Li", "Mn"),
)

Using Materials Project MACE for MACECalculator with /home/colin/.cache/mace/MACE_MPtrj_20229model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using head Default out of ['Default']


/home/colin/miniforge/envs/tc312/lib/python3.12/site-packages/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Evaluating CE vs MACE:   0%|          | 0/60 [00:00<?, ?it/s]

: 